# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [117]:
# Write your code below.
%reload_ext dotenv
%dotenv 


cannot find .env file


In [118]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd



+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [131]:
import os
import sys
from glob import glob
import time 
# Write your code below.

PRICE_DATA = os.getenv("PRICE_DATA")

# Use glob to find the path of all parquet files in the directory PRICE_DATA
parquet_files = glob(os.path.join(PRICE_DATA, "*/*.parquet"))
parquet_files


['../../05_src/data/prices\\A\\A_2013.parquet',
 '../../05_src/data/prices\\A\\A_2014.parquet',
 '../../05_src/data/prices\\A\\A_2015.parquet',
 '../../05_src/data/prices\\A\\A_2016.parquet',
 '../../05_src/data/prices\\A\\A_2017.parquet',
 '../../05_src/data/prices\\A\\A_2018.parquet',
 '../../05_src/data/prices\\A\\A_2019.parquet',
 '../../05_src/data/prices\\A\\A_2020.parquet',
 '../../05_src/data/prices\\A\\A_2021.parquet',
 '../../05_src/data/prices\\A\\A_2022.parquet',
 '../../05_src/data/prices\\A\\A_2023.parquet',
 '../../05_src/data/prices\\A\\A_2024.parquet',
 '../../05_src/data/prices\\ABNB\\ABNB_2020.parquet',
 '../../05_src/data/prices\\ABNB\\ABNB_2021.parquet',
 '../../05_src/data/prices\\ABNB\\ABNB_2022.parquet',
 '../../05_src/data/prices\\ABNB\\ABNB_2023.parquet',
 '../../05_src/data/prices\\ABNB\\ABNB_2024.parquet',
 '../../05_src/data/prices\\ABT\\ABT_2013.parquet',
 '../../05_src/data/prices\\ABT\\ABT_2014.parquet',
 '../../05_src/data/prices\\ABT\\ABT_2015.parquet'

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [135]:
# Write your code below.

import numpy as np

dd_px = dd.read_parquet(parquet_files).set_index("ticker")


In [163]:
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1=x['Close'].shift(1))
).assign(
    returns=lambda x: x['Close'] / x['Close_lag_1'] - 1
).assign(
    hi_lo_range=lambda x: x['High'] - x['Low']
))


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [170]:
# Write your code below.
# Convert the Dask data frame to a pandas data frame.
df = dd_feat.compute()


In [172]:
# Add a rolling average return calculation with a window of 10 days.
# ip: Consider using .rolling(10).mean().

df['rolling_avg_return'] = df.groupby('ticker')['returns'].rolling(window=10).mean().reset_index(level=0, drop=True)

dd_feat = dd_feat.persist()

dd_feat.head()



,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year,Close_lag_1,returns,hi_lo_range
ticker,,,,,,,,,,,,
A,2013-12-02 00:00:00-05:00,34.995823,35.126720,34.760205,34.825653,2039962,0.0,0.0,2013,NaN,NaN,0.366515
A,2013-12-03 00:00:00-05:00,34.635851,34.799476,34.400232,34.642395,3462706,0.0,0.0,2013,34.825653,-0.005262,0.399244
A,2013-12-04 00:00:00-05:00,34.583481,35.231432,34.504944,35.067806,3377288,0.0,0.0,2013,34.642395,0.012280,0.726488
A,2013-12-05 00:00:00-05:00,34.917279,35.211801,34.766746,35.015453,2530939,0.0,0.0,2013,35.067806,-0.001493,0.445055
A,2013-12-06 00:00:00-05:00,35.185629,35.951385,35.185629,35.885937,4268513,0.0,0.0,2013,35.015453,0.024860,0.765756


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

Was it necessary to convert to pandas to calculate the moving average return?

No, it was not necessary to convert to pandas to calculate the moving average return. Dask supports rolling window operations directly.

b) Yes, it would have been better to do it in Dask if you are dealing with large datasets that do not fit into memory. Dask is designed to handle operations on large datasets by breaking them into smaller chunks and processing them in parallel. This makes it more efficient for large-scale data processing compared to pandas, which requires the entire dataset to fit into memory.



## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.